# NTDS'18 milestone 1: network collection and properties
[Effrosyni Simou](https://lts4.epfl.ch/simou), [EPFL LTS4](https://lts4.epfl.ch)

## Students

* Team: `32`
* Students: `George Adaimi, Okan Altingovde, Isinsu Katircioglu, Sena Kiciroglu`
* Dataset: `Free Music Archive`

## Rules

* Milestones have to be completed by teams. No collaboration between teams is allowed.
* Textual answers shall be short. Typically one to three sentences.
* Code has to be clean.
* You cannot import any other library than we imported.
* When submitting, the notebook is executed and the results are stored. I.e., if you open the notebook again it should show numerical results and plots. We won't be able to execute your notebooks.
* The notebook is re-executed from a blank state before submission. That is to be sure it is reproducible. You can click "Kernel" then "Restart & Run All" in Jupyter.

## Objective 

The purpose of this milestone is to start getting acquainted to the network that you will use for this class. In the first part of the milestone you will import your data using [Pandas](http://pandas.pydata.org) and you will create the adjacency matrix using [Numpy](http://www.numpy.org). This part is project specific. In the second part you will have to compute some basic properties of your network. **For the computation of the properties you are only allowed to use the packages that have been imported in the cell below.** You are not allowed to use any graph-specific toolboxes for this milestone (such as networkx and PyGSP). Furthermore, the aim is not to blindly compute the network properties, but to also start to think about what kind of network you will be working with this semester. 

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Library Added
import ast

#in order to compute the feature distances and form the adjacency matrix:
from scipy.spatial.distance import pdist, squareform 


## Part 1 - Import your data and manipulate them. 

###  A. Load your data in a Panda dataframe.

First, you should define and understand what are your nodes, what features you have and what are your labels. Please provide below a Panda dataframe where each row corresponds to a node with its features and labels. For example, in the the case of the Free Music Archive (FMA) Project, each row of the dataframe would be of the following form:


| Track   |  Feature 1  | Feature 2 | . . . | Feature 518|  Label 1 |  Label 2 |. . .|Label 16|
|:-------:|:-----------:|:---------:|:-----:|:----------:|:--------:|:--------:|:---:|:------:|
|         |             |           |       |            |          |          |     |        |

It is possible that in some of the projects either the features or the labels are not available. This is OK, in that case just make sure that you create a dataframe where each of the rows corresponds to a node and its associated features or labels.

In [ ]:
# Your code here.
def load(filename):

    if 'features' in filename:
        return pd.read_csv(filename, index_col=0, header=[0, 1, 2])

    if 'echonest' in filename:
        return pd.read_csv(filename, index_col=0, header=[0, 1, 2])

    if 'genres' in filename:
        return pd.read_csv(filename, index_col=0)

    if 'tracks' in filename:
        tracks = pd.read_csv(filename, index_col=0, header=[0, 1])

        COLUMNS = [('track', 'tags'), ('album', 'tags'), ('artist', 'tags'),
                   ('track', 'genres'), ('track', 'genres_all')]
        for column in COLUMNS:
            tracks[column] = tracks[column].map(ast.literal_eval)

        COLUMNS = [('track', 'date_created'), ('track', 'date_recorded'),
                   ('album', 'date_created'), ('album', 'date_released'),
                   ('artist', 'date_created'), ('artist', 'active_year_begin'),
                   ('artist', 'active_year_end')]
        for column in COLUMNS:
            tracks[column] = pd.to_datetime(tracks[column])

        SUBSETS = ('small', 'medium', 'large')
        tracks['set', 'subset'] = tracks['set', 'subset'].astype(
                pd.api.types.CategoricalDtype(SUBSETS, ordered=True))

        COLUMNS = [('track', 'genre_top'), ('track', 'license'),
                   ('album', 'type'), ('album', 'information'),
                   ('artist', 'bio')]
        for column in COLUMNS:
            tracks[column] = tracks[column].astype(pd.api.types.CategoricalDtype())

    return tracks

def separate_GenreID(tracks, features):
    genres = load('dataset/genres.csv')
    genres_simplify = genres[['title','#tracks']].sort_values('#tracks',ascending = 0)
    genres_toFilter = ['Hip-Hop','Instrumental', 'Rock']
    genres_ids = genres_simplify[genres_simplify['title'].isin(genres_toFilter)].index
    genres_ids = genres_ids.tolist()
    chosen_tracks = tracks[('track', 'genres_all')].to_frame()[[not (set(genres_ids).isdisjoint(set(x))) for x in tracks[('track', 'genres_all')].values]]
    
    merged = pd.merge(features, chosen_tracks, on="track_id")
    
    return merged

tracks = load('dataset/tracks.csv')
features = load('dataset/features.csv') # Read features.csv

small = tracks[tracks['set', 'subset'] == 'small'] #chooses tracks that are in the subset 'small'
subset = small[(small['track', 'genre_top'] == 'Hip-Hop') | (small['track', 'genre_top'] == 'Rock')] # filters the small subset to the ones that are Hip-Hop and Rock
merged = pd.merge(features, subset['track', 'genre_top'].to_frame(), on="track_id") # Merges the subset frame with the feature frame based on 'track_id'
small_features = features[features.index.isin(subset.index)] # Filters the tracks in the features dataframe based on the subset dataframe

### B. Create the adjacency matrix of your network.

Remember that there are edges connecting the attributed nodes that you organized in the dataframe above. The connectivity of the network is captured by the adjacency matrix $W$. If $N$ is the number of nodes, the adjacency matrix is an $N \times N$ matrix where the value of $W(i,j)$ is the weight of the edge connecting node $i$ to node $j$.  

There are two possible scenarios for your adjacency matrix construction, as you already learned in the tutorial by Benjamin:

1) The edges are given to you explicitly. In this case you should simply load the file containing the edge information and parse it in order to create your adjacency matrix. See how to do that in the  [graph from edge list]() demo.

2) The edges are not given to you. In that case you will have to create a feature graph. In order to do that you will have to chose a distance that will quantify how similar two nodes are based on the values in their corresponding feature vectors. In the [graph from features]() demo Benjamin showed you how to build feature graphs when using Euclidean distances between feature vectors. Be curious and explore other distances as well! For instance, in the case of high-dimensional feature vectors, you might want to consider using the cosine distance. Once you compute the distances between your nodes you will have a fully connected network. Do not forget to sparsify by keeping the most important edges in your network.

Follow the appropriate steps for the construction of the adjacency matrix of your network and provide it in the Numpy array ``adjacency`` below: 

In [ ]:
# Your code here
NUM_OF_SAMPLES = len(small_features)

features_part = small_features.loc[:,'mfcc']#.iloc[:NUM_OF_SAMPLES] # Take mfcc column and only first NUM_OF_SAMPLES (For testing)
distances = pdist(features_part.values, metric='cosine') # Used cosine since high dimensional (Need to check other distances)
kernel_width = distances.mean()
weights = np.exp(-distances**2 / kernel_width**2) # Same as in notebook

#check histogram to see which values are common and where we can set the threshold
fig = plt.figure()
plt.hist(weights)
plt.title('Distribution of weights')
plt.show()
plt.close(fig)

weights[weights < 0.95] = 0 # Set values less than 0.6 to 0
adjacency = squareform(weights)
#n_nodes = # the number of nodes in the network


## Part 2

Execute the cell below to plot the (weighted) adjacency matrix of your network.

In [ ]:
fig = plt.figure()
plt.spy(adjacency, markersize=0.01)
plt.title('adjacency matrix')
plt.show()
plt.close(fig)

fig = plt.figure()
plt.imshow(adjacency)
plt.title('adjacency matrix')
plt.show()
plt.close(fig)

### Question 1

What is the maximum number of links $L_{max}$ in a network with $N$ nodes (where $N$ is the number of nodes in your network)? How many links $L$ are there in your collected network? Comment on the sparsity of your network.

In [ ]:
# Your code here.
# The maximum number of links is found through N*(N-1)/2
n_nodes = len(features_part)
L_max = n_nodes*(n_nodes-1)/2

# The number of links in our current network is the amount of non-zero values in our weights vector.
L = np.count_nonzero(weights)

#The ratio of L/L_max
ratio = 100* L/L_max
print("L_max: ", L_max, ", L: ", L , ", Ratio: ", ratio)

**Your answer here.**
* L_max = 
* L = 
* Sparsity: Our network has 4.45% of the maximum amount of links. We can say that our graph, like most real world graphs, is sparse.

### Question 2

Is your graph directed or undirected? If it is directed, convert it to an undirected graph by symmetrizing the adjacency matrix.

**Your answer here.**
Our graph is undirected, since the feature distance of track i to track j is the same as the feature distance of track j to track i.

In [ ]:
# Your code here.
#Our graph is already undirected!

### Question 3

In the cell below save the features dataframe and the **symmetrized** adjacency matrix. You can use the Pandas ``to_csv`` to save the ``features`` and Numpy's ``save`` to save the ``adjacency``. We will reuse those in the following milestones.

In [ ]:
# Your code here.
np.save("adjacency.npy", adjacency)
features.to_csv("our_features.csv")

### Question 4

Are the edges of your graph weighted?

**Your answer here.**
The edges of our graph are weighted. The higher weights between two songs mean that they have similar feature vectors. Lower weights mean that they are less similar.

### Question 5

What is the degree distibution of your network? 

In [ ]:
degree = np.sum(adjacency, axis = 0) # Your code here. It should be a numpy array.
assert degree.shape[0] == n_nodes

Execute the cell below to see the histogram of the degree distribution.

In [ ]:
weights = np.ones_like(degree) / float(n_nodes)
plt.hist(degree, weights=weights);

What is the average degree?

In [ ]:
# Your code here.
ave_degree = np.sum(degree)/n_nodes
print(ave_degree)

### Question 6

Comment on the degree distribution of your network.

In [ ]:
#print log plot of degree distribution
fig = plt.figure()
n, bins, patches = plt.hist(degree, weights=weights, bins = 200)
x_ax = bins[1:]
plt.close(fig)

fig = plt.figure()
plt.suptitle("degree distribution")
plt.subplot(121)
plt.title("normal plot")
plt.plot(n, x_ax,  "ro")
plt.subplot(122)
plt.title("log plot")
plt.semilogy(n, x_ax, "ro")
plt.show()
plt.close(fig)

**Your answer here.**
I don't want to comment before all our answers are final yet but here's a tentative answer.

* The degree distribution seems to be a power-law distribution, judging from the log-plot. 

* Our average degree is 344.4 which is quite large considering that small world networks have an average degree of 6.

### Question 7

Write a function that takes as input the adjacency matrix of a graph and determines whether the graph is connected or not.

In [ ]:
def connected_graph(adjacency):
    """Determines whether a graph is connected.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    bool
        True if the graph is connected, False otherwise.
    """
    
    # Your code here.
    
    return connected

Is your graph connected? Run the ``connected_graph`` function to determine your answer.

In [ ]:
# Your code here.

### Question 8

Write a function that extracts the connected components of a graph.

In [ ]:
def find_components(adjacency):
    """Find the connected components of a graph.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    
    Returns
    -------
    list of numpy arrays
        A list of adjacency matrices, one per connected component.
    """
    
    # Your code here.
    
    return components

How many connected components is your network composed of? What is the size of the largest connected component? Run the ``find_components`` function to determine your answer. 

In [ ]:
# Your code here.

### Question 9

Write a function that takes as input the adjacency matrix and a node (`source`) and returns the length of the shortest path between that node and all nodes in the graph using Dijkstra's algorithm. **For the purposes of this assignment we are interested in the hop distance between nodes, not in the sum of weights. **

Hint: You might want to mask the adjacency matrix in the function ``compute_shortest_path_lengths`` in order to make sure you obtain a binary adjacency matrix. 

In [ ]:
def compute_shortest_path_lengths(adjacency, source):
    """Compute the shortest path length between a source node and all nodes.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    
    Returns
    -------
    list of ints
        The length of the shortest path from source to all nodes. Returned list should be of length n_nodes.
    """
    
    # Your code here.
    
    return shortest_path_lengths

### Question 10

The diameter of the graph is the length of the longest shortest path between any pair of nodes. Use the above developed function to compute the diameter of the graph (or the diameter of the largest connected component of the graph if the graph is not connected). If your graph (or largest connected component) is very large, computing the diameter will take very long. In that case downsample your graph so that it has 1.000 nodes. There are many ways to reduce the size of a graph. For the purposes of this milestone you can chose to randomly select 1.000 nodes. 

In [ ]:
# Your code here.

### Question 11

Write a function that takes as input the adjacency matrix, a path length, and two nodes (`source` and `target`), and returns the number of paths of the given length between them.

In [ ]:
def compute_paths(adjacency, source, target, length):
    """Compute the number of paths of a given length between a source and target node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    source: int
        The source node. A number between 0 and n_nodes-1.
    target: int
        The target node. A number between 0 and n_nodes-1.
    length: int
        The path length to be considered.
    
    Returns
    -------
    int
        The number of paths.
    """
    # Your code here.
    adjacency_l_pow = np.linalg.matrix_power(adjacency, length)
    n_paths = adjacency_l_pow[source, target]
    return n_paths

Test your function on 5 pairs of nodes, with different lengths.

In [ ]:
print(compute_paths(adjacency, 0, 10, 1))
print(compute_paths(adjacency, 0, 10, 2))
print(compute_paths(adjacency, 0, 10, 3))
print(compute_paths(adjacency, 23, 67, 2))
print(compute_paths(adjacency, 15, 93, 4))

### Question 12

How many paths of length 3 are there in your graph? Hint: calling the `compute_paths` function on every pair of node is not an efficient way to do it.

In [ ]:
# Your code here.

### Question 13

Write a function that takes as input the adjacency matrix of your graph (or of the largest connected component of your graph) and a node and returns the clustering coefficient of that node. 

In [ ]:
def compute_clustering_coefficient(adjacency, node):
    """Compute the clustering coefficient of a node.
    
    Parameters
    ----------
    adjacency: numpy array
        The (weighted) adjacency matrix of a graph.
    node: int
        The node whose clustering coefficient will be computed. A number between 0 and n_nodes-1.
    
    Returns
    -------
    float
        The clustering coefficient of the node. A number between 0 and 1.
    """
    
    # Your code here.
    li = adjacency[0:node, 0:node].sum() + adjacency[node:, 0:node].sum() + adjacency[0:node, node:].sum() + adjacency[node:, node:].sum()
    ki = adjacency[node,:].sum()
    clustering_coefficient = li/((ki)*(ki-1))
    return clustering_coefficient


### Question 14

What is the average clustering coefficient of your graph (or of the largest connected component of your graph if your graph is disconnected)? Use the function ``compute_clustering_coefficient`` to determine your answer.

In [ ]:
# Your code here.
n_node = adjacency.shape[0]
clustering_coeff_list = [compute_clustering_coefficient(adjacency, n) for n in range(0, n_node)]
avg_clustering_coeff = np.array(clustering_coeff_list).mean()
print(avg_clustering_coeff)